In [3]:
!pip install xmltodict

In [4]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

import pandas as pd
import xmltodict
import json

##### 공공데이터활용지원센터_보건복지부 코로나19 감염 현황
> 활용신청하기
https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043376

In [7]:
key='c9VRxUXOrgEoo9RlRqdxfK%2F%2BQYEU7cH7eALxP3LXcvBUTllj2gYOeh2%2Fik1nAUmEBMSEDHqQd%2F%2BlAbBKtQ6dNg%3D%3D'
url = f'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey={key}&'
queryParams = urlencode({ quote_plus('pageNo') : 1, 
                          quote_plus('numOfRows') : 10,
                          quote_plus('startCreateDt') : '20220301',
                          quote_plus('endCreateDt') : '20220321'})
url2 = url + queryParams
response = urlopen(url2)
# print(type(response)) # HTTPSresponse 
results = response.read().decode("utf-8")
# print(type(results))   # str
results_to_json = xmltodict.parse(results)
data = json.loads(json.dumps(results_to_json))
#print(type(data))   # dic
print(data)   

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'createDt': '2022-03-21 09:10:38.912', 'deathCnt': '12757', 'decideCnt': '9582560', 'seq': '824', 'stateDt': '20220321', 'stateTime': '00:00', 'updateDt': '2022-03-22 08:57:25.86'}, {'createDt': '2022-03-20 09:22:09.424', 'deathCnt': '12428', 'decideCnt': '9373395', 'seq': '823', 'stateDt': '20220320', 'stateTime': '00:00', 'updateDt': '2022-03-22 08:58:21.437'}, {'createDt': '2022-03-19 09:20:10.848', 'deathCnt': '12101', 'decideCnt': '9038706', 'seq': '822', 'stateDt': '20220319', 'stateTime': '00:00', 'updateDt': '2022-03-22 08:58:40.104'}, {'createDt': '2022-03-18 09:06:55.679', 'deathCnt': '11782', 'decideCnt': '8657271', 'seq': '821', 'stateDt': '20220318', 'stateTime': '00:00', 'updateDt': '2022-03-22 08:59:00.784'}, {'createDt': '2022-03-17 09:28:28.783', 'deathCnt': '11481', 'decideCnt': '8250360', 'seq': '820', 'stateDt': '20220317', 'stateTime': '00:00', 'updateDt': '20

위의 데이터를 분석해서 아래의 형태의 CSV 파일을 만들자
```
날짜,누적확진자,격리해제환자,치료중환자,사망자수
20210119,73104,59468,12353,1283
20210120,73508,60180,12028,1300

```

Understand that most of the markup languages (XML, HTML, JSON, YAML, etc) have a similar encapsulation (or nested) structure. Most of the time the payloads are in a list like format (called line items in business). 

Once the overall structure of the data package have been identified. it is most likely that it would have `[key]:[value], [key]:[value], [key]:[value], ... `format which is a list format we have learnt with numpy. So just convert the list into a DataFrame. Than relabel the columns.


In [8]:
lists = data['response']['body']['items']['item']
df = pd.DataFrame(lists)
ndf = df[['stateDt', 'decideCnt', 'deathCnt']]
ndf.columns=['날짜','누적확진자','사망자수']

In [9]:
ndf.to_csv('covid19.csv', index=False)

## Additional: 
For evaluating JSON payloads, Copy the above JSON return "`{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'accDefRate': '1.4573418044', ... }`" and to format with https://www.freeformatter.com/json-formatter.html 



```
{
   "response": {
      "header": {
         "resultCode": "00",
         "resultMsg": "NORMAL SERVICE."
      },
      "body": {
         "items": {
            "item": [
               {
                  "accDefRate": "1.4573418044",
                  "accExamCnt": "5192119",
                  "accExamCompCnt": "5043978",
                  "careCnt": "12028",
                  "clearCnt": "60180",
                  "createDt": "2021-01-20 09:38:16.549",
                  "deathCnt": "1300",
                  "decideCnt": "73508",
                  "examCnt": "148141",
                  "resutlNegCnt": "4970470",
                  "seq": "392",
                  "stateDt": "20210120",
                  "stateTime": "00:00",
                  "updateDt": "2021-04-20 15:23:25.562"
               },
               {
                  "accDefRate": "1.4661941407",
                  "accExamCnt": "5140315",
                  "accExamCompCnt": "4985970",
                  "careCnt": "12353",
                  "clearCnt": "59468",
                  "createDt": "2021-01-19 09:39:49.588",
                  "deathCnt": "1283",
                  "decideCnt": "73104",
                  "examCnt": "154345",
                  "resutlNegCnt": "4912866",
                  "seq": "391",
                  "stateDt": "20210119",
                  "stateTime": "00:00",
                  "updateDt": "2021-04-20 15:23:34.538"
               }
            ]
         },
         "numOfRows": "10",
         "pageNo": "1",
         "totalCount": "2"
      }
   }
}
```

